In [52]:
import time
from dateutil import rrule
from datetime import datetime, timedelta
import unittest

"""
Это шедулер. На вход принимает callback, который будет вызываться с аргументами *args, **kwargs, pattern в cron формате или
freq - частота вызова в секундах, и init_run - запускаться ли при первом вызове.
Этот шедулер не совсем полноценный: предполагается, что нужно самостоятельно вызывать функцию check(), а шедулер лишь проверит, что
время пришло и в этом случае вызовет callback и запланирует время следующего запуска.
"""

class ScheduleTimer():
    def __init__(self, callback, pattern=None, freq=None, init_run=False, *args, **kwargs):
        """
        :param pattern: (seconds) (minutes) (hours) (day) (month) (day-of-week) in cron-like manner: 0 5 * * * * ... means every 5 minute of every hour 
        :param freq: `datetime.timedelta`
        :param init_run: bool. Run or init or not.
        """
        self.callback = callback
        self.next_call = None
        self.init_run = init_run

        self.args = args or ()
        self.kwargs = kwargs or {}

        if freq is not None:
            self.pattern = self._freq_to_pattern(freq)
        else:
            self.pattern = pattern
        units = sec, min, hou, day, mon, dow = [self._parse_time_unit(units) for units in self.pattern.split()]
        freqs = [rrule.SECONDLY, rrule.MINUTELY, rrule.HOURLY, rrule.DAILY, rrule.MONTHLY, rrule.WEEKLY]
        param_names = ['bysecond', 'byminute', 'byhour', 'bymonthday', 'bymonth', 'byweekno']

        try:
            freq = freqs[units.index(None)]
        except:
            # not stars at all
            freq = rrule.YEARLY

        self.rrule_params = dict(zip(param_names, units))
        self.rrule_params['freq'] = freq
        if not self.init_run:
            self._upd_next_call()

    def _freq_to_pattern(self, freq):
        """
        Построение cron строки по freq вынесем в отдельную функцию, чтобы было проще читать __init__
        """
        freq_sec = round(freq.total_seconds())

        # cast pattern by frequency
        if freq >= timedelta(days=1):
            # freq is supported only for hour and less frequency
            raise Exception('Too large frequency: {}. Use frequency less than a day'.format(freq_sec))

        if freq_sec < 0:
            raise Exception('Bad frequency: {}. Frequency should strictly greater than zero!'.format(freq_sec))

        units = [1, 60, 3600, 3600 * 24]

        # find which unit is used
        unit_i = 0
        while True:
            if units[unit_i] <= freq_sec < units[unit_i + 1]:
                break
            unit_i += 1

        if units[unit_i + 1] % freq_sec != 0:
            raise Exception('Bad frequency: {}'.format(freq_sec))

        freq_units = int(freq_sec / units[unit_i])
        freq_vals = [i * freq_units for i in range(units[unit_i + 1] // freq_sec)]
        freq_vals = map(str, freq_vals)

        pattern_vals = []
        for i in range(6):
            if i < unit_i:
                pattern_vals.append('0')
            elif i == unit_i:
                if freq_units != 1:
                    pattern_vals.append(','.join(freq_vals))
                else:
                    pattern_vals.append('*')
            elif i > unit_i:
                pattern_vals.append('*')
        pattern = ' '.join(pattern_vals)
        print(pattern)
        return pattern
        
    def _parse_time_unit(self, unit):
        if unit == '*':
            return None
        unit = unit.split(',')
        return [int(x) for x in unit]

    def _upd_next_call(self):
        # set next_call to closest
        self.last_call_dt = datetime.fromtimestamp(self.next_call) if self.next_call else datetime.now()
        next_call_lst = list(rrule.rrule(dtstart=self.last_call_dt + timedelta(seconds=1), 
                                         count=1,
                                         **self.rrule_params))

        if not next_call_lst:
            raise Exception('Bad scheduler input!')

        self.next_call = next_call_lst[0].timestamp()

    def check(self):
        if self.init_run or time.time() > self.next_call:
            self.run()
            self.init_run = False
            self._upd_next_call()

    def run(self):
        return self.callback(*self.args, **self.kwargs)

class TestStringMethods(unittest.TestCase):
    def callback(self, param):
        param.append(1)
        
    def test_pattern_hours(self):
        scheduler = ScheduleTimer(self.callback, None, freq=timedelta(hours=4), init_run=True)
        self.assertEqual(scheduler.pattern, '0 0 0,4,8,12,16,20 * * *')

    def test_pattern_minutes(self):
        scheduler = ScheduleTimer(self.callback, None, freq=timedelta(minutes=20), init_run=True)
        self.assertEqual(scheduler.pattern, '0 0,20,40 * * * *')

    def test_pattern_seconds(self):
        scheduler = ScheduleTimer(self.callback, None, freq=timedelta(seconds=10), init_run=True)
        self.assertEqual(scheduler.pattern, '0,10,20,30,40,50 * * * * *')

    def test_freq_over_day(self):
        with self.assertRaisesRegex(Exception, 'Too large frequency: (\d+). Use frequency less than a day'):
            scheduler = ScheduleTimer(self.callback, None, freq=timedelta(days=2))

    def test_divisible_freq(self):
        with self.assertRaisesRegex(Exception, 'Bad frequency: (\d+)'):
            scheduler = ScheduleTimer(self.callback, None, freq=timedelta(minutes=35))

    def test_init_run(self):
        param = []
        scheduler = ScheduleTimer(self.callback, None, freq=timedelta(seconds=2), init_run=True, param=param)
        scheduler.check()
        self.assertEqual(len(param), 1)

    def test_without_init_run(self):
        param = []
        scheduler = ScheduleTimer(self.callback, None, freq=timedelta(seconds=2), param=param)
        scheduler.check()
        self.assertEqual(len(param), 0)
    
    def test_pattern(self):
        param = []
        scheduler = ScheduleTimer(self.callback, "0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38," +
                                  "40,42,44,46,48,50,52,54,56,58 * * * * *", param=param)
        scheduler.check()
        time.sleep(2)
        scheduler.check()
        self.assertEqual(len(param), 1)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


...

0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58 * * * * *


.....

0 0 0,4,8,12,16,20 * * *
0 0,20,40 * * * *
0,10,20,30,40,50 * * * * *
0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58 * * * * *



----------------------------------------------------------------------
Ran 8 tests in 2.011s

OK
